In [4]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [5]:
name = 'glove-100-angular'
nameFull = name +'-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (1183514, 100)
testDataset :  (10000, 100)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [6]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = 100
    time_start = perf_counter()
    index = indexMethod(faiss.IndexFlatL2(d), d, M, faiss.METRIC_L2)
    index.train(datasetImages)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexIVFFlat, datasetTrainImages)

Took 1.567 seconds


In [7]:
# (min, max) = createIndexNumerous(createIndex, AnnoyIndex, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [8]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 2.658 seconds


In [9]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n','max : ', max,)

In [10]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [11]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [12]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [13]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

636623 || 660281
4.3422 || 3.8792
977260 || 93438
4.3827 || 3.8838
567634 || 1133489
4.4382 || 3.9021
362125 || 97478
4.4461 || 3.9037
108638 || 995275
4.4476 || 3.9294
235112 || 207076
4.4543 || 3.9378
637400 || 989330
4.5116 || 3.9523
563875 || 846101
4.5185 || 3.955
669881 || 242797
4.5231 || 3.9568
570338 || 402216
4.5277 || 3.9697


In [14]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.0843
0.2897
0.9414


In [15]:
print(indexes[0]) # will find the 100 nearest neighbors

[ 636623  977260  567634  362125  108638  235112  637400  563875  669881
  570338  747047 1014096  140992  449254   33080  982764  312780  612575
  512629  514528  364453  759786  138599 1097397  186633 1159130  357982
  123179     852  497394  700861 1097900  418892  557143  998688  345052
  756339  411027   98828  107976  212181  431385  752628 1002599  576925
  970217 1025673  295718  597018  820420  836361 1138059  571834  764781
  271050  987252  956450  841793  212933  317337  419626  656064 1172041
 1018098  598700 1117741  642822  948503  747124 1060647  538228 1109384
    8926  137241 1004337  974565  538293 1181415  885218  342720  149637
  295610  559920  501354  602541  561958 1021373 1047148  502145 1155590
  350379  154013  907140  594194 1061135  580945 1154045    7425  967032
  612118]


In [16]:
# draw_mnist(indexes, distances, datasetTrainImages)